In [3]:
import pandas as pd

In [4]:
user_file_path = 'base_users.csv'
repo_file_path ='base_repositories.csv'  

In [40]:
usersdf = pd.read_csv(user_file_path)
repositoriesdf = pd.read_csv(repo_file_path)


In [6]:
# usersdf['hireable'].fillna(False, inplace=True)
# usersdf.to_csv('users.csv', index=False)

### Q1
Who are the top 5 users in Austin with the highest number of followers? List their login in order, comma-separated.

In [7]:
top_users = usersdf.nlargest(5, 'followers')
top_user_logins = top_users['login'].tolist()
top_user_logins

['getify', 'benawad', 'steveklabnik', 'cloudflare', 'jbogard']


### Q2
Who are the 5 earliest registered GitHub users in Austin? List their login in ascending order of created_at, comma-separated.

In [11]:
earliest_users = usersdf.sort_values(by='created_at')
earliest_logins = earliest_users['login'].head(5).tolist()
earliest_logins

['jnewland', 'joshknowles', 'hassox', 'jicksta', 'dan']

### Q3
What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [12]:
popular_licenses = repositoriesdf['license_name'].dropna()
license_counts = popular_licenses.value_counts()
top_3_licenses = license_counts.nlargest(3)
result = top_3_licenses.index.tolist()
result

['mit', 'apache-2.0', 'other']

### Q4
Which company do the majority of these developers work at?

In [16]:
company_counts = usersdf['company'].value_counts()
most_common_company = company_counts.idxmax()
# most_common_count = company_counts.max()
most_common_company

'GOOGLE'

### Q5
Which programming language is most popular among these users?

In [17]:
language_counts = repositoriesdf['language'].value_counts()
most_popular_language = language_counts.idxmax()
most_popular_count = language_counts.max()
most_popular_language

'JavaScript'

### Q6
Which programming language is the second most popular among users who joined after 2020?

In [18]:
usersdf['created_at'] = pd.to_datetime(usersdf['created_at'])
recent_users = usersdf[usersdf['created_at'] > '2020-01-01']
recent_user_logins = recent_users['login']
filtered_repositories = repositoriesdf[repositoriesdf['login'].isin(recent_user_logins)]

language_counts = filtered_repositories['language'].value_counts()

if len(language_counts) >= 2:
    second_most_popular_language = language_counts.index[1]
    second_most_popular_count = language_counts[1]
    print(f"The second most popular programming language is: {second_most_popular_language} ({second_most_popular_count} repositories)")
else:
    print("Not enough data to determine the second most popular language.")

The second most popular programming language is: HTML (75 repositories)


### Q7
Which language has the highest average number of stars per repository?

In [19]:
average_stars = repositoriesdf.groupby('language')['stargazers_count'].mean()

highest_average_language = average_stars.idxmax()
highest_average_count = average_stars.max()
highest_average_language


'Fennel'

### Q8
Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [21]:
usersdf['leader_strength'] = usersdf['followers'] / (1 + usersdf['following'])
top_leaders = usersdf.nlargest(5, 'leader_strength')
top_logins = top_leaders['login'].tolist()
', '.join(top_logins)

'getify, cloudflare, benawad, oracle, ContinuumIO'

### Q9
What is the correlation between the number of followers and the number of public repositories among users in Austin?

In [22]:
correlation = usersdf['followers'].corr(usersdf['public_repos'])
round(correlation, 3)

0.151

### Q10
Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.
Regression slope of followers on repos (to 3 decimal places, e.g. 0.123 or -0.123)

In [23]:
import statsmodels.api as sm

In [24]:
X = usersdf['public_repos'] 
y = usersdf['followers']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

slope = model.params['public_repos']
round(slope, 3)

4.102

### Q11
Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?
Correlation between projects and wiki enabled (to 3 decimal places, e.g. 0.123 or -0.123)

In [27]:
repositoriesdf['has_projects'] = repositoriesdf['has_projects'].astype(int)
repositoriesdf['has_wiki'] = repositoriesdf['has_wiki'].astype(int)

correlation = repositoriesdf['has_projects'].corr(repositoriesdf['has_wiki'])
round(correlation, 3)

0.274

### Q12
Do hireable users follow more people than those who are not hireable?
Average of following per user for hireable=true minus the average following for the rest (to 3 decimal places, e.g. 12.345 or -12.345)

In [28]:
avg_following_hireable = usersdf[usersdf['hireable'] == True]['following'].mean()
avg_following_non_hireable = usersdf[usersdf['hireable'] == False]['following'].mean()

difference = avg_following_hireable - avg_following_non_hireable
round(difference, 3)

nan

### Q13
Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)
Regression slope of followers on bio word count (to 3 decimal places, e.g. 12.345 or -12.345)

In [43]:
users_with_bios = usersdf[usersdf['bio'].notna()]

users_with_bios.loc[:, 'bio_word_count'] = users_with_bios['bio'].apply(lambda x: len(x.split()))

X = users_with_bios['bio_word_count']  
y = users_with_bios['followers'] 

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
slope = model.params['bio_word_count']

round(slope, 3)

C:\Users\abhim\AppData\Local\Temp\ipykernel_19344\3335181940.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_bios.loc[:, 'bio_word_count'] = users_with_bios['bio'].apply(lambda x: len(x.split()))


7.974

In [39]:
bios = usersdf['bio'].dropna()
', '.join(bios)

'Kyle Simpson is a Human-Centric Technologist. He\'s fighting for the people behind the pixels., React.js, Typescript, Node.js, PostgreSQL, All i\'m trying to do is enjoy every day to the fullest with my Pokmon., The Barley Architect, Architect Consultant, Building 𝝠 Expo • Follow me on 𝕏 for updates 🥓, Open Source at Oracle, Full Professor at The University of Texas at Austin working on \r\nSpatial Data Analytics, Geostatistics and Machine Learning, Advanced data processing, analysis, and visualization tools for Python & R., Co-founder of Luro. Lead developer at Paravel. Co-host of ShopTalkShow. Purveyor of tiny jQueries., Ruby core contributor. Learn how to contribute to open source with my app CodeTriage (free) and my book How to Open Source (paid), links below 👇🏻, CEO, Fleet.  Creator & BDFL, sails.js, <script>alert(1)</script>, Tech lead of Cloudflare Workers, Cap\'n Proto, Sandstorm.io., Developer at Bluesky, Staff Engineer at @discord working on Design Systems and Accessibility.

### Q14
Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [30]:
repositoriesdf['created_at'] = pd.to_datetime(repositoriesdf['created_at'])
repositoriesdf['day_of_week'] = repositoriesdf['created_at'].dt.dayofweek

weekend_repos = repositoriesdf[(repositoriesdf['day_of_week'] == 5) | (repositoriesdf['day_of_week'] == 6)]
repo_counts = weekend_repos['login'].value_counts()

top_users = repo_counts.nlargest(5).index.tolist()

', '.join(top_users)

'FellowTraveler, realityexpander, OR13, PaulBratslavsky, skeptycal'

### Q15
Do people who are hireable share their email addresses more often?
[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

In [31]:
total_users = usersdf.shape[0]

hireable_with_email = usersdf[(usersdf['hireable'] == True) & (usersdf['email'].notna())].shape[0]
total_hireable = usersdf[usersdf['hireable'] == True].shape[0]

non_hireable_with_email = usersdf[(usersdf['hireable'] == False) & (usersdf['email'].notna())].shape[0]
total_non_hireable = usersdf[usersdf['hireable'] == False].shape[0]

fraction_hireable = hireable_with_email / total_hireable if total_hireable > 0 else 0
fraction_non_hireable = non_hireable_with_email / total_non_hireable if total_non_hireable > 0 else 0

difference = fraction_hireable - fraction_non_hireable
round(difference, 3)

0.515

### Q16
Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)
Number of users with the most common surname

In [32]:
users_with_names = usersdf[usersdf['name'].notna()]['name'].str.strip()
surnames = users_with_names.apply(lambda x: x.split()[-1])

surname_counts = surnames.value_counts()

most_common_count = surname_counts.max()
most_common_surnames = surname_counts[surname_counts == most_common_count].index.tolist()

most_common_surnames.sort()
# number_of_users = most_common_count

', '.join(most_common_surnames)


'Labs, Moore, Smith'